In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 500)

In [2]:
!ls

02. Классификация текстов-Copy1.ipynb
20news-bydate
Examples
sentiment labelled sentences
smsspamcollection
ДЗ_№1.ipynb


In [3]:
!ls "smsspamcollection"

SMSSpamCollection readme


### 1-2 Загрузили и прочитали данные 

In [4]:
ds = pd.read_csv("smsspamcollection/SMSSpamCollection", 
                 delimiter="\t", names=("is_spam", "text"))

In [5]:
ds.head()

,is_spam,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


### 3. Преобразуем типы ham (не спам) и spam в 0 и 1 соответственно

In [6]:
for i in range(ds.is_spam.size):
    if ds.is_spam[i] == "ham":
        ds.is_spam[i] = 0
    else:
        ds.is_spam[i] = 1

In [7]:
ds.head()

,is_spam,text
0,0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives around here though"


### 4. Получим матрицу признаков X

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(ds.text)

In [9]:
vectorizer.get_feature_names()

['00',
 '000',
 '000pes',
 '008704050406',
 '0089',
 '0121',
 '01223585236',
 '01223585334',
 '0125698789',
 '02',
 '0207',
 '02072069400',
 '02073162414',
 '02085076972',
 '021',
 '03',
 '04',
 '0430',
 '05',
 '050703',
 '0578',
 '06',
 '07',
 '07008009200',
 '07046744435',
 '07090201529',
 '07090298926',
 '07099833605',
 '07123456789',
 '0721072',
 '07732584351',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07781482378',
 '07786200117',
 '077xxx',
 '078',
 '07801543489',
 '07808',
 '07808247860',
 '07808726822',
 '07815296484',
 '07821230901',
 '078498',
 '07880867867',
 '0789xxxxxxx',
 '07946746291',
 '0796xxxxxx',
 '07973788240',
 '07xxxxxxxxx',
 '08',
 '0800',
 '08000407165',
 '08000776320',
 '08000839402',
 '08000930705',
 '08000938767',
 '08001950382',
 '08002888812',
 '08002986030',
 '08002986906',
 '08002988890',
 '08006344447',
 '0808',
 '08081263000',
 '08081560665',
 '0825',
 '083',
 '0844',
 '08448350055',
 '08448714184',
 '0845',
 '08450542832',
 '084

In [10]:
#print(ds.iloc[0].text)

In [11]:
#print(X[0])

In [12]:
feature_names = np.array(vectorizer.get_feature_names())
feature_names[X[0].nonzero()[1]]

array(['wat', 'amore', 'got', 'there', 'cine', 'buffet', 'la', 'world',
       'great', 'bugis', 'in', 'only', 'available', 'crazy', 'point',
       'jurong', 'until', 'go'], 
      dtype='<U34')

### Построим линейный классификатор и посчитаем точность определения тональности

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

cls = LogisticRegression()
res = cross_val_score(cls, X, list(ds["is_spam"]), scoring="f1", cv=10)
print(np.mean(res))

0.932640298361


### Ответ на 5 вопрос: 
### 0.932640298361

In [14]:
sample = vectorizer.transform(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", "FreeMsg: Txt: claim your reward of 3 hours talk time", "Have you visited the last lecture on physics?", "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", "Only 99$"])

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

#c = 10
cls = LogisticRegression()
cls.fit(X, list(ds["is_spam"]))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
for i in range(cls.predict(sample).size):
    print(cls.predict(sample)[i], end=" ")

1 1 0 0 0 

### Ответ на 6 вопрос: 
### 1 1 0 0 0

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def cls_dep_on_ngram_CV(x,y):
    vectorizer = CountVectorizer(ngram_range=(x,y))
    X = vectorizer.fit_transform(ds.text)
    cls = LogisticRegression()
    res = cross_val_score(cls, X, list(ds["is_spam"]), scoring="f1", cv=10)
    print(str(np.mean(res))[:4], end=" ")

In [18]:
cls_dep_on_ngram_CV(2,2)
cls_dep_on_ngram_CV(3,3)
cls_dep_on_ngram_CV(1,3)

0.82 0.72 0.92 

### Ответ на 7 вопрос: 
### 0.82 0.72 0.92 

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

def cls_dep_on_ngram_MNB(x,y):
    vectorizer = CountVectorizer(ngram_range=(x,y))
    X = vectorizer.fit_transform(ds.text)

    cls = MultinomialNB()
    res = cross_val_score(cls, X, list(ds["is_spam"]), scoring="f1", cv=10)
    print(str(np.mean(res))[:4], end=" ")

In [20]:
cls_dep_on_ngram_MNB(2,2)
cls_dep_on_ngram_MNB(3,3)
cls_dep_on_ngram_MNB(1,3)

0.64 0.37 0.88 

### Ответ на 8 вопрос: 
### 0.64 0.37 0.88

### TF*IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(ds.text)

cls = LogisticRegression()
res = cross_val_score(cls, X_tfidf, list(ds["is_spam"]), scoring="f1", cv=5)
print(str(np.mean(res))[:4])

0.83


### Ответ на 9 вопрос: 
### при использовании CountVectorizer получили 0.92, при использовании TfidfVectorizer, следовательно качество валидации понизилось.

# 10 задание

In [68]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 500)

In [69]:
ds = pd.read_csv("smsspamcollection/SMSSpamCollection", 
                 delimiter="\t", names=("is_spam", "text"))

In [70]:
for i in range(ds.is_spam.size):
    if ds.is_spam[i] == "ham":
        ds.is_spam[i] = 0
    else:
        ds.is_spam[i] = 1

In [71]:
ds.head()

,is_spam,text
0,0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives around here though"


#### Токенизация

In [72]:
import nltk
from nltk.tokenize import word_tokenize

#создать список слов из предложений
sent_ls = [word_tokenize(str(ds.text[i])) for i in range(ds.is_spam.size)]


#### Лемматизируем слова

In [8]:
print(sent_ls[0])

['Go', 'until', 'jurong', 'point', ',', 'crazy..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'got', 'amore', 'wat', '...']


from nltk.stem import WordNetLemmatizer
sent_ls_lem = [[]for i in range(ds.is_spam.size)]
lemmatizer = WordNetLemmatizer()
for i in range(ds.is_spam.size):
    sent_ls_lem[i] = [lemmatizer.lemmatize(w).lower() for w in sent_ls[i]]
            

#### Стэмминг (помог лучше, нежели лемматизация)

In [43]:
from nltk.stem import PorterStemmer
sent_ls_lem = [[]for i in range(ds.is_spam.size)]
ps = PorterStemmer()
for i in range(ds.is_spam.size):
    sent_ls_lem[i] = [ps.stem(w).lower() for w in sent_ls[i]]

#### Удалим stop words (не дали особого прироста)

from nltk.corpus import stopwords
stop_engl = [w for w in stopwords.words("english")]

for i in range(ds.is_spam.size):
    for w in sent_ls_lem[i]:
        if w in stop_engl:
            sent_ls_lem[i].pop(sent_ls_lem[i].index(w))

In [44]:
final_sent = [[] for i in range(ds.is_spam.size)]
for i in range(ds.is_spam.size):
    final_sent[i] = " ".join(sent_ls_lem[i])

In [45]:
#print(final_sent)

In [46]:
ds.text = pd.Series(final_sent, index=[i for i in range(ds.is_spam.size)])

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def cls_dep_on_ngram_CV1(x,y):
    vectorizer = CountVectorizer(ngram_range=(x,y))
    X = vectorizer.fit_transform(ds.text)
    for c in [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]:
    #c = 15000
        cls = LogisticRegression(C=c)
        res = cross_val_score(cls, X, list(ds["is_spam"]), scoring="f1", cv=10)
        print(c, str(np.mean(res)))
    print("=====")

In [67]:
cls_dep_on_ngram_CV1(2,2)
cls_dep_on_ngram_CV1(3,3)
cls_dep_on_ngram_CV1(1,3)

0.01 0.0568859983011
0.1 0.700931884916
0.5 0.814005726312
1 0.827171312456
5 0.842795752959
10 0.847998509228
100 0.854983806201
200 0.859315482498
500 0.867970952774
1000 0.870515627515
10000 0.875086746128
15000 0.875259845375
20000 0.877825143534
100000 0.882012739986
=====


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-sco

0.01 0.0
0.1 0.454481811821
0.5 0.713951179728
1 0.72599797769
5 0.743142598683
10 0.748283146937
100 0.758813639009
200 0.76376029746
500 0.775787936601
1000 0.780705294902
10000 0.804872797047
15000 0.806809780034
20000 0.808607956265
100000 0.811517179504
=====
0.01 0.819184152068
0.1 0.909787445305
0.5 0.923693455103
1 0.924432153025
5 0.92670108577
10 0.928289438085
100 0.930526348114
200 0.929787161219
500 0.930604154683
1000 0.932068496033
10000 0.932807193955
15000 0.932807193955
20000 0.933590014224
100000 0.933590014224
=====


#### Используя CountVectorizer со стандартными параметрами

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def cls_dep_CV_gen():
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(ds.text)
    for c in [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]:
        cls = LogisticRegression(C=c)
        res = cross_val_score(cls, X, list(ds["is_spam"]), scoring="f1", cv=10)
        print(c, str(np.mean(res)))
    print("=====")

In [76]:
cls_dep_CV_gen()

0.01 0.783702308103
0.1 0.904201830786
0.5 0.92667510292
1 0.932640298361
5 0.939999527925
10 0.938478680146
100 0.939771313827
200 0.939751204828
500 0.939771313827
1000 0.940431217304
10000 0.939763925051
15000 0.940462582893
20000 0.940462582893
100000 0.943449680811
=====


# Итоговый вывод:

Проделав даную работу, можно убедиться в том, что на данном типе текстов TfidfVectorizer работает хуже, чем CountVectorizer. 
Так же применение Стемминга оказалось немного лучше в итоге, чем применение Лемматизации. Удаление stop words не дали почти никакого эффекта. В преобразованном тексте лучший результат выдал CountVectorizer со стандартными параметрами, нежели с применением N-грамм (разница очень небольшая)
### Таким образом, лучший результат получился 0,94